In [1]:
from glob import glob
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
import os

In [2]:
pdfs = glob('./췌장암_pdf/*.pdf')

loaders = [PyMuPDFLoader(pdf) for pdf in pdfs]
docs = []
for loader in loaders:
    docs.extend(loader.load())

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500
)
splits = text_splitter.split_documents(docs)

/opt/homebrew/anaconda3/envs/new_okay/lib/python3.9/site-packages/langchain_community/document_loaders/parsers/pdf.py:300: UserWarning: Warning: Empty content on page 0 of document ./췌장암_pdf/200207 국민암예방수칙 실천지침_췌장암.pdf
  warnings.warn(
/opt/homebrew/anaconda3/envs/new_okay/lib/python3.9/site-packages/langchain_community/document_loaders/parsers/pdf.py:300: UserWarning: Warning: Empty content on page 28 of document ./췌장암_pdf/췌장암_대한종양내과학회.pdf
  warnings.warn(


In [3]:
print(f"문서 수: {len(docs)}")
print(f"첫 문서 내용: {docs[0].page_content if docs else '문서 없음'}")

문서 수: 130
첫 문서 내용: 췌장암
암에 대한 바른 이해



In [4]:
load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large', chunk_size=500)
vectordb = Chroma(embedding_function=embedding, collection_name="pancreatic_cancer", persist_directory="./pancreatic_chroma_db")

In [5]:
vectordb.add_documents(splits)

['6d76339d-626e-4495-a3d5-53a95b2c6399',
 '72ad855a-b294-4a77-94f2-d0523ca57d66',
 'e181a2ab-a1a6-4bf9-8d7e-25b6ffb370cc',
 'bfe48533-08e5-4ba2-97bd-bc772f24dd46',
 '9bd87918-2529-4541-b8ad-ff418fdace3b',
 'd4dd6035-7e28-41ec-a76f-33394605ae08',
 '8eb384e7-087d-4da3-bf20-21e62d58b732',
 'c8c6bbba-bc46-48bf-9391-68819932b6f7',
 '49d3cee5-c4c7-4c33-8c9d-0f1686450915',
 '09cb7b26-d09f-490c-94d1-7933df23d1e7',
 '64d2cac6-eb18-413b-8075-c5d6f98ff0bf',
 '17b29511-f250-4206-acb2-8af0c7458740',
 '813dde35-16f7-4dc4-ab0c-8cc055fb94dc',
 '79e54e41-0653-4363-ad4d-24115da131d5',
 'a40d0d25-ae1c-4470-b28d-08aae1c0b2ad',
 'e6c4f3c1-2419-4a7b-b512-7114ea788390',
 '82f4310f-fd4d-491f-97ca-6a4058441263',
 '8a4b36eb-e6cc-4cf7-9f7a-7bbef962da80',
 'd6983a9e-4598-46c1-8408-2c8358cf104d',
 '9e95faa2-f374-4cbd-8d10-7fb42ff0a45d',
 '55b6a484-0396-432c-ad53-3ddcf3f968f9',
 '16e7dc1d-a875-4363-90ba-b00b0340af24',
 '33d6e2c2-2e92-4b9b-b0eb-c973dd546cde',
 'c71ad201-49c5-4042-b009-fc342949a10d',
 '588d342a-07c7-

In [6]:
llm = ChatOpenAI(temperature=0, model='gpt-4o-mini')

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(search_kwargs={"k": 3}),
    llm=llm
)

In [7]:
template = """
Your response should be in JSON format.
You are an empathetic chatbot designed to provide information and support regarding diseases.
You aim to maintain a consistent and friendly style while ensuring that explanations are easy for anyone to understand.
When explaining your symptoms and how to deal with them, please answer clearly and accurately. Depending on the situation, use jargon to explain, but don't lie, be precise and detailed.
You will do best to provide appropriate answers to questions.
Your goal is to provide the answers I seek and offer the assistance.
At the end of each response, You'll provide key terms related to the question, including diseases and medications.
Put a line break after the period.
Please refrain from unnecessary words.
Please answer me only once.
don't say it over and over again
Don't use repeated phrases.
Don't rewrite the question at the end.
Don't answer anything after writing the keywords.
Don't add questions at the end.
Stop answering after listing keyword words.
Please answer in 1000 words or less.
Please must reply in Korean.
you are designed to output json.


Answer the question based only on the following context: {context}

Question: {input}
Output Format (JSON)
{{
"question": "Write the original user-submitted questions.",
"answer": "Full response to original question.",
"sources": "When writing your answer, organize it into sentences and record the context in which you wrote it.",
"source_documents": [
    {{
      "title": "The title of the source document.",
      "page": "The page number of the source document.",
      "content": "The relevant content extracted from the source document.",
      "url": "The URL or file path of the source document."
    }}]
}}
"""

prompt = ChatPromptTemplate.from_template(template)

In [8]:
setup_and_retrieval = RunnableParallel(
    {"context": retriever_from_llm, "input": RunnablePassthrough()}
)

output_parser = StrOutputParser()

chain = setup_and_retrieval | prompt | llm | output_parser

In [9]:
input_query = "췌장암의 원인은 무엇인가요?"
response = chain.invoke(input_query)
print(response)

```json
{
  "question": "췌장암의 원인은 무엇인가요?",
  "answer": "췌장암의 발병 원인은 여러 가지 요인이 복합적으로 작용합니다. 가장 강력한 발암 물질로 알려진 것은 담배이며, 흡연은 췌장암의 상대 위험도를 2∼5배 증가시킵니다. 또한, 고지방 및 육류 소비가 췌장암 발생과 관련이 있을 가능성이 높습니다. 반면, 과일, 채소, 식이 섬유소, 비타민 C는 췌장암 발생 억제 효과가 보고되고 있습니다. 췌장암은 45세 이상에서 발생하기 쉬우며, 장기간의 당뇨병력, 흡연력, 고지질 식이 등이 위험 요인으로 작용합니다. 만성 췌장염 또한 췌장암 발생의 고위험군으로 분류되며, 만성 췌장염 환자에서 췌장암 발생 가능성은 약 3%로 알려져 있습니다. 유전적 소인, 과음, 비만, 유해 화학물질 노출 등도 췌장암의 위험 요인으로 작용할 수 있습니다.",
  "sources": "이 답변은 췌장암의 원인에 대한 여러 연구 결과를 바탕으로 작성되었습니다.",
  "source_documents": [
    {
      "title": "췌장암",
      "page": "10",
      "content": "췌장암의 발병원인은 무엇인가요? 췌장암의 가장 강력한 발암 물질로 알려진 것은 담배입니다. 흡연의 췌장암 상대 위험도는 2∼5배로 현재까지 알려진 가장 중요한 위험인자입니다. 췌장암의 발생이 선진국에서 높은 것을 고려할 때 음식물 중 지방과 육류의 소비가 췌장암의 발생과 관련이 있을 가능성이 높고 과일, 채소, 식이 섬유소, 비타민 C 등이 췌장암 발생에 대한 억제 효과가 보고되고 있습니다.",
      "url": "./췌장암_pdf/160818_췌장암.pdf"
    },
    {
      "title": "췌장암",
      "page": "2",
      "content": "췌장암이 발생하기 쉬운 요인에는 45세 이상, 장기간의 당뇨병력, 흡연력, 두경부, 폐 및 방광암의 과거력, 고지질 식이 등이 있으며 최

췌장암의 발병 원인은 여러 가지 요인이 복합적으로 작용합니다. 가장 강력한 발암 물질로 알려진 것은 담배이며, 흡연은 췌장암의 상대 위험도를 2∼5배 증가시킵니다. 또한, 고지방 및 육류 소비가 췌장암 발생과 관련이 있을 가능성이 높습니다. 반면, 과일, 채소, 식이 섬유소, 비타민 C는 췌장암 발생 억제 효과가 보고되고 있습니다. 췌장암은 45세 이상, 장기간의 당뇨병력, 흡연력, 고지질 식이, 만성 췌장염, 유전적 소인 등 다양한 위험 요인에 의해 발생할 수 있습니다. 특히, 만성 췌장염은 췌장암 발생의 고위험군으로 간주되며, 만성 췌장염 환자에서 췌장암 발생 가능성은 약 3%로 알려져 있습니다. 이 외에도 유해 화학물질에 노출되는 것도 위험 요인으로 작용할 수 있습니다.

In [10]:
print(chain.invoke('췌장암 1기와 2기 3기 차이를 알려주세요'))

```json
{
  "question": "췌장암 1기와 2기 3기 차이를 알려주세요",
  "answer": "췌장암의 병기는 1기, 2기, 3기로 나뉘며 각 단계는 암의 진행 정도에 따라 다릅니다.\n\n1기: 이 단계에서는 암이 췌장에만 국한되어 있으며, 전이가 없습니다. 수술을 통해 췌장만 절제하여 암을 치료할 수 있는 가장 초기 상태입니다.\n\n2기: 암이 췌장 바깥으로 퍼져 주변 장기(예: 십이지장, 담관)나 림프절로 전이되기 시작하지만, 주요 동맥 혈관에는 침범하지 않은 상태입니다. 이 경우에도 수술을 통해 치료할 수 있는 가능성이 있습니다.\n\n3기: 이 단계에서는 암이 주요 동맥 혈관을 침범하여 국소적으로 진행되고, 수술이 불가능한 상태입니다. 암이 주변 장기로 퍼져 있지만, 아직 먼 장기로의 전이는 없습니다.",
  "sources": "췌장암의 병기와 특징에 대한 정보는 다양한 의료 자료를 바탕으로 정리되었습니다.",
  "source_documents": [
    {
      "title": "췌장암의 진단",
      "page": "7",
      "content": "췌장암의 병기 결정은 널리 쓰이는 TNM 분류법을 따릅니다. T(tumor)는 종양의 크기와 침윤 정도를 나타내고, N(node)은 주위 림프절로 퍼진 정도를, M(metastasis)은 다른 장기로의 전이 여부를 나타내며, 이들 세 요소를 조합하여 병기를 1~4기로 구분합니다.",
      "url": "./췌장암_pdf/160818_췌장암.pdf"
    },
    {
      "title": "췌장암의 특징과 병기",
      "page": "2",
      "content": "췌장암은 보통 초기에는 증상이 별로 없는 경우가 많아서 진행이 된 상태에서 진단되는 경우가 많습니다. 병의 범위는 병기(stage)로 나타납니다.",
      "url": "./췌장암_pdf/췌장암_대한종양내과학회.pdf"
  

췌장암의 병기는 암의 진행 정도에 따라 1기, 2기, 3기로 구분됩니다. \n\n1기: 암이 췌장에만 국한되어 있으며, 전이가 없는 상태입니다. 이 경우 수술을 통해 암을 치료할 수 있습니다. \n\n2기: 암이 췌장 주변 장기로 퍼지기 시작하지만, 주요 동맥 혈관에는 침범하지 않은 상태입니다. 이 단계에서도 수술이 가능할 수 있습니다. \n\n3기: 암이 주요 동맥 혈관을 침범하여 국소적으로 진행된 상태로, 수술이 불가능한 경우입니다. 이 단계에서는 치료 방법이 제한적입니다.